In [33]:
import cv2 as cv


In [6]:
capture = cv.VideoCapture('output7.mp4')
count = 0
path = 'C:/Users/LENOVO/Documents/uni/4t/psiv/tracking/psiv_trackers/frames/'
while ( capture.isOpened()):
    ret, frame = capture.read()
    if (ret == True):
        cv.imwrite(path +  'img_%010d.jpg' % count, frame)
        count += 1
        if cv.waitKey(1) == ord('s'):
            break
    else:
        break
capture.release()
cv.destroyAllWindows()

#guardar imagene en un archivo

In [36]:
capture = cv.VideoCapture('output7.mp4')
fgbg = cv.bgsegm.createBackgroundSubtractorMOG()





while ( capture.isOpened()):
    ret, frame = capture.read()
    if (ret == False): break
       
    fgmaskMOG = fgbg.apply(frame)
    
    
    
    
    
    cv.imshow('MOG', fgmaskMOG)
    cv.imshow('frame', frame)
    
    
    if cv.waitKey(1) == ord('q'):
            break
    
    
capture.release()
cv.destroyAllWindows()


KeyboardInterrupt: 

In [23]:
capture.release()
cv.destroyAllWindows()